In [1]:
%pip install langchain litellm requests chromadb sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# https://huggingface.co/datasets/fka/awesome-chatgpt-prompts/viewer/default/train?p=1&row=135 <---- Cool Prompts
# https://api.together.xyz/playground
from langchain.chat_models import ChatLiteLLM
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
import os

In [22]:
os.environ['TOGETHERAI_API_KEY'] = "API_KEY"
chat = ChatLiteLLM(model="together_ai/mistralai/Mistral-7B-Instruct-v0.1", max_tokens=4086)

In [4]:
system_message = """
I want you to act as a career counselor. I will provide you with an individual looking for guidance in their professional life, and your task is to help them determine what careers they are most suited for based on their skills, interests and experience. You should also conduct research into the various options available, explain the job market trends in different industries and advice on which qualifications would be beneficial for pursuing particular fields."
"""

resume_file = open("resume.txt", "r", encoding="utf-8")
resume_text = resume_file.read()
resume_file.close()

user_message = f"I'm looking to break into tech, what are some of the top recommendations you would have based on my resume: <resume_start> {resume_text}<resume_end>"
template = PromptTemplate.from_template("""
<s>[INST] {system_message} [/INST] Model answer</s>[INST] {user_question} [/INST]                                                                          
""")

prompt = template.format(system_message=system_message, user_question=user_message)

tokens = chat.get_num_tokens(prompt)
print(tokens)
response = chat.predict(prompt)
print(response)

/home/bphil/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1642 > 1024). Running this sequence through the model will result in indexing errors


1642
Based on your resume, it appears that you have a strong background in IT and cloud computing, with experience in a variety of roles including Solutions Architect, Technical Account Manager, and IT Support Engineer. Your experience with Amazon Web Services (AWS) is particularly notable, as it is a highly sought-after skill in the tech industry.

Given your background, I would recommend exploring career paths in cloud computing and software engineering. Some specific roles to consider include:

1. Cloud Solutions Architect: As a Cloud Solutions Architect, you would be responsible for designing and implementing cloud-based solutions for clients. This role requires a deep understanding of cloud technologies and the ability to design and deploy complex systems.
2. DevOps Engineer: DevOps Engineers are responsible for automating the software development process, including testing, deployment, and monitoring. This role requires a strong background in software development and a deep under

In [4]:
def role_query(system, file_name, user):
    system_message = system

    transcript_file = open(file_name, "r", encoding="utf-8")
    transcript_text = transcript_file.read()
    transcript_file.close()

    user_question = f" {user}: {transcript_text}"
    prompt = template.format(
        system_message=system_message, user_question=user_question)

    tokens = chat.get_num_tokens(prompt)
    print(tokens)
    response = chat.predict(prompt)
    print(response)




In [54]:
system = """
I want you to act as a social media influencer. You will create content for various platforms such as Instagram, Twitter or YouTube and engage with followers in order to increase brand awareness and promote products or services.
"""
file_name = "transcript_2.txt"
user_question = "Please repurpose the following youtube transcript for twitter"
role_query(system,file_name,user_question)

3342
Hey guys, it's a big news day because Activision Blizzard has officially been bought by Microsoft. The trailer for Overwatch, Fallout, and Starfield has been released. I think it's great that Microsoft is investing in these games and I'm excited to see what happens. What are your thoughts on this news? #Microsoft #ActivisionBlizzard #Gaming #Overwatch #Fallout #Starfield


In [56]:
system = """
I want you to act as a salesperson. Try to market something to me, but make what you're trying to market look more valuable than it is and convince me to buy it. Now I'm going to pretend you're calling me on the phone and ask what you're calling for.
"""
file_name = "transcript_2.txt"
user_question = "Please use the following transcript to create a sells conversation"
role_query(system,file_name,user_question)

3359
Hello, I'm calling to tell you about the exciting news that Activision Blizzard has been acquired by Microsoft. This is a massive deal in the gaming industry, and it's great to see that Microsoft is investing in gaming.

As a result of this acquisition, Activision Blizzard's CEO, Bobby Kotick, will be stepping down on January 1st, 2024. However, he will remain with the company until then and will continue to oversee the gaming division.

There is a lot of speculation about what Bobby Kotick will do next. Some people think he may take on a separate leadership role at Microsoft, while others think he may focus on a metaverse project. Whatever he does, it's clear that he has made a significant impact on the gaming industry and will continue to do so in the future.

In my opinion, this acquisition is a great move for Microsoft. It shows their commitment to the gaming industry and their willingness to invest in it. With Activision Blizzard's vast library of games and franchises, Micros

In [5]:
# Get recent information related to Austin scott, does mistral know about him?
who_is = chat.predict("Who is James Austin Scott")
print(who_is)


James Austin Scott is an American actor and producer. He is best known for his role as Lucas Scott on the television series "The Office (US)" and "Days of Our Lives." He has also appeared in several films, including "The Secret Life of Walter Mitty" and "The Perfect Wife."


In [6]:
# It actually doesn't know, let's go ahead and get some information on him.
import requests
from langchain.document_transformers import BeautifulSoupTransformer
from langchain.text_splitter import HTMLHeaderTextSplitter
from langchain.docstore.document import Document

# Initalize BS Transformer
bs_transformer = BeautifulSoupTransformer()

url = "https://en.wikipedia.org/wiki/Austin_Scott_(politician)"

# Get the information on Austin Scott
response = requests.get(url)
text = response.text

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#Put it in a document
doc = Document(page_content=text, metadata={"source": url})

# Clean the text
transformed_doc = bs_transformer.transform_documents([doc], tags_to_extract= ["h1", "h2", "h3", "h4" "p", "ul", "table", "figure"])

# Try cleaning with HTMLsplitter and Raw text
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(text)
print(len(html_header_splits))
# Slice the document up
chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
splits = text_splitter.split_documents(html_header_splits)
print(len(splits))


35
95


In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/bphil/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(splits, hf)

In [42]:

# Add documents
db.add_documents(splits)

['2acfbffa-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc0e0-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc13a-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc180-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc1c6-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc20c-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc252-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc28e-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc2d4-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc310-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc356-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc392-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc3ce-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc414-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc45a-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc496-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc4dc-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc518-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc554-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc59a-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc5e0-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc61c-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc662-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc69e-6a2d-11ee-9b46-00155de4a1fb',
 '2acfc6e4-6a2d-

In [41]:
db.delete_collection()

In [43]:
docs = db.similarity_search("Who is James Austin Scott")
print(len(docs))


1


In [46]:
# Format prompt how Mistral likes it
question = "What causes does Austin Scott support?"
template = PromptTemplate.from_template("""
<s>[INST] Use this additional context to help answer the User question: {system_message} [/INST] Model answer</s>[INST] User Question: {user_question} [/INST]                                                                          
""")

retriever = db.as_retriever()
docs = retriever.get_relevant_documents(question)
print(len(docs))
context = ""
for d in docs:
    if chat.get_num_tokens(context) <= 3000:
        context += d.page_content
    else:
        break
    
prompt = template.format(system_message=context, user_question=question)

tokens = chat.get_num_tokens(prompt)
print(tokens)
who_is = chat.predict(prompt)
print(who_is)

2
230
Austin Scott is an American politician who represents the 12th district of Georgia in the United States House of Representatives. He is a member of the Republican Party and was first elected in 2010. Scott is known for his support of various causes, including agriculture, veterans' affairs, and national security.

In particular, Scott has been a strong advocate for the Agriculture Improvement Act of 2018, also known as the Farm Bill. He has spoken extensively about the importance of this legislation, which provides funding for agricultural research, conservation, and rural development programs. Scott has also been a vocal supporter of the Veterans Health Care Integration Act, which aims to improve access to healthcare for veterans.

In addition to these specific causes, Scott has also been a strong advocate for national security and has spoken out against foreign threats to the United States. He has been a member of the House Armed Services Committee, which has jurisdiction over 